Variación del número de qubits del subsistema A, nA y del peso w. Realizamos ya el circuito completo, donde la primera vuelta sería como tener un solo bloque (donde calculamos el número de Schmidt), y luego escogemos el número de reticiones (lo llamo v, en este cógido está puesto v=2 dentro del bucle)

In [ ]:
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info import Statevector, Operator
from qiskit.quantum_info import DensityMatrix, partial_trace
from qiskit.quantum_info import schmidt_decomposition
from qiskit.result import marginal_counts
from qiskit.primitives import StatevectorSampler, StatevectorEstimator

from qiskit.primitives import Sampler
from qiskit.visualization import plot_histogram
from qiskit.visualization import array_to_latex
from qiskit.visualization import latex
from qiskit.visualization import plot_bloch_vector
from qiskit.visualization import plot_bloch_multivector
from qiskit.visualization import visualize_transition


from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp


import matplotlib.pyplot as plt
import matplotlib.ticker as tck
from scipy.linalg import logm, expm


from matplotlib import cm
from matplotlib.ticker import LinearLocator

In [ ]:
def lista(n):

    di={}

    lista=[]

    for i in range(2**n):
        lista1=[]
        lista2=[]
        s=0
        for j in range(n):
            if str(format(i,'b').zfill(n))[j]=='1':
                s+=1
        if s==2:
            for j in range(n):
                if str(format(i,'b').zfill(n))[j]=='0':
                    lista1+='I'
                elif str(format(i,'b').zfill(n))[j]=='1':
                    lista1+='Z'
            lista2+=lista1[0]
            for j in range(1,n):
                lista2[0]+=lista1[j]
            di[str(format(i,'b').zfill(n))]=lista2[0]


    for i in range(2**n):
        if str(format(i, 'b').zfill(n)) in di:
            lista += tuple((di[str(format(i,'b').zfill(n))],))

    return lista

In [ ]:
#Grupo A: nA qubits que se miden al salir de la puerta 
#Grupo B: nB qubits que reaprovechamos para el siguiente circuito al salir de la puerta conjunta


number_of_phases =41
phases_beta= np.linspace(0, 2 * np.pi, number_of_phases)
phases_gamma=np.linspace(0, 2 * np.pi, number_of_phases)

w=np.linspace(0,2 * np.pi,number_of_phases)
#w=np.linspace(np.pi/4, 1.5, number_of_phases)
n=7
nA=np.arange(1,7,1) #número qubits sistema A, de 1 a 5 (1,2,3,4,5) qubits
#nA=np.arange(3,5,1)

################Funciones#####################################################
def S(rho):
        return -((np.matmul(rho,logm(rho))).trace())


#Número de Schmidt:############################################################

def sch(rho):
    sch=0
    #uno de los subsistemas es [0], osea un solo qubit:
    #Using this convention, setting qargs=[0] will partition the state as |q3q2q1>|q0>
    
    a=schmidt_decomposition(rho,list(range(j)))
    
    for i in range(len(a)):
        if a[i][0]!=1e-10:
            sch+=1
    return sch

##########Cálculo entropía con schmidt decomposition#################################

def S_sch(rho,nA):

    k=schmidt_decomposition(rho,list(range(nA)))
    S_sch=0
    for i in range(len(k)):
        lambd= (k[i][0])**2
        S_sch+=-lambd*(np.log(lambd))
    
    return S_sch

##############################################################################

y=np.zeros((len(w),len(nA)))  #entropía total
y_A=np.zeros((len(w),len(nA))) #entropía subsistema A
y_B=np.zeros((len(w),len(nA))) #entropía subsistema B
ym=np.zeros((len(w),len(nA))) #entropía mutua
yc_AB=np.zeros((len(w),len(nA))) #entropía condicionada A sobre B
yc_BA=np.zeros((len(w),len(nA))) #entropía condicionada B sobre A

for j in nA:

    m=int(n*(n-1)/2) #número de uniones
    
    nB=int(n-j)
    
    for i in w:

        l=[]
        for h in range(m):
            t=()
            strg=lista(int(n))[h]
            t+=(strg,i)
            l.append(t)

        hamiltonian = SparsePauliOp.from_list(l)

        ansatz = QAOAAnsatz(hamiltonian, reps=1)    
        
        circuit=ansatz.assign_parameters([0,0.5])   #beta= 0 y gamma =0.5

        rho=DensityMatrix(circuit)

        print('1ª vuelta: número de Schmidt',sch(rho),'para un peso', i, 'para un nº de qubits en el sistema A',j)
        
        
        rho_A=partial_trace(DensityMatrix(circuit), qargs=list(range(j,j+nB))) #Matriz reducida sistema A
        rho_B=partial_trace(DensityMatrix(circuit), qargs=list(range(j))) #Matriz reducida sistema B



#####################Varias REPETICIONES ##############################

        v=2 #número de vueltas 

        for q in range(1,v+1): #q va de 1 a v: 1,2,3,..,v

            strg=''
            for g in range(j):
                strg+='0'
        
            cero_A=Statevector.from_label(strg)
            rho_A=DensityMatrix(cero_A)

            #rho=np.kron(rho_A,rho_B)
            rho=rho_B.tensor(rho_A)
            
            print(rho)
            

            rho_A=partial_trace(rho.evolve(circuit), qargs=list(range(j,j+nB))) #Matriz reducida sistema A
            rho_B=partial_trace(rho.evolve(circuit), qargs=list(range(j))) #Matriz reducida sistema B


    ############Entropía############################

        b=nA.tolist().index(j)
        a=w.tolist().index(i)
        
        y[a][b]=np.real(S(rho))
        y_A[a][b]=np.real(S(rho_A))
        y_B[a][b]=np.real(S(rho_B))

        print('entropía subsistema A y B:',y_A[a][b],y_B[a][b])
    
########Entropía mutua###########################

        Sm=S(rho_A)+S(rho_B)-S(rho)

        ym[a][b]=np.real(Sm)
    
##########Entropía condicional###################

        Sc_AB= S(rho)-S(rho_B)# rho_0 sobre rho_1
        Sc_BA= S(rho)-S(rho_A) # rho_1 sobre rho_0

        yc_AB[a][b]=np.real(Sc_AB)
        yc_BA[a][b]=np.real(Sc_BA)


In [ ]:
from matplotlib.colors import LogNorm

%matplotlib inline

plt.clf()

z=np.array(nA)
x=np.array(w) 

c = plt.pcolormesh(z, x, y_A, cmap ='Greens')
plt.colorbar(c)

plt.ylabel('Peso w de uno de los ZZ (el resto son 1)')
plt.xlabel('qubits')

plt.title('Entropía Subsistema A', fontweight ="bold")

#Nos excusamos de representar la entropía del subsistema B porque, matemáticamente,
#son iguales si el estado total es puro



plt.pause(0.01)

a = plt.pcolormesh(z, x, y_B, cmap ='Blues')
plt.colorbar(a)

plt.title('Entropía Subsistema B', fontweight ="bold")
plt.ylabel('Peso w')
plt.xlabel('qubits subsistema A')

plt.pause(0.01)

b= plt.pcolormesh(z, x, ym, cmap ='Blues')
plt.colorbar(b)

plt.title('Entropía mutua', fontweight ="bold")



plt.pause(0.01)

d=plt.pcolormesh(z, x, yc_AB, cmap ='Greens')
plt.colorbar(d)

plt.title('Entropía condicionada A sobre B', fontweight ="bold")
